In [ ]:
import numpy as np
from numpy.core.fromnumeric import shape
import os

In [ ]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes

nusc = NuScenes(version='v1.0-mini', dataroot='/home/max/projects/masterthesis/mini_nuscenes', verbose=True)

In [ ]:
# Load last 3 point clouds
#############################3
# Print List of scenes
nusc.list_scenes()

#Get set of scenes
scenes = nusc.scene
#Get first scenes
scene_0 = scenes[0]
# Get token of first frame
first_sample_token = scene_0['first_sample_token']
sample_0 = nusc.get('sample', first_sample_token)
# Get tokens for 2 following frames
second_sample_token = sample_0['next']
sample_1 = nusc.get('sample', second_sample_token)
third_sample_token = sample_1['next']
sample_2 = nusc.get('sample', third_sample_token)

In [ ]:
sample_1

In [ ]:
# Get LIDAR pointcloud
sensor = 'LIDAR_TOP'
lidar_top_data_0 = nusc.get('sample_data', sample_0['data'][sensor])
lidar_top_data_0

In [ ]:
# Get LIDAR KF pointcloud
lidar_top_data_1 = nusc.get('sample_data', sample_1['data'][sensor])
assert lidar_top_data_1['is_key_frame'] == True
print(lidar_top_data_1)

# Get LIDAR KF pointcloud
lidar_top_data_2 = nusc.get('sample_data', sample_2['data'][sensor])
assert lidar_top_data_2['is_key_frame'] == True
lidar_top_data_2

In [ ]:
# nusc.render_sample_data(lidar_top_data_0['token'])
# nusc.render_sample_data(lidar_top_data_1['token'])
# nusc.render_sample_data(lidar_top_data_2['token'])

In [ ]:
# Get point clouds
import os.path as osp
from nuscenes.utils.data_classes import LidarPointCloud, Box

pcl0_path = osp.join(nusc.dataroot, lidar_top_data_0['filename'])
pcl1_path = osp.join(nusc.dataroot, lidar_top_data_1['filename'])
pcl2_path = osp.join(nusc.dataroot, lidar_top_data_2['filename'])
#Load Pointclouds
pc0 = LidarPointCloud.from_file(pcl0_path)
pc1 = LidarPointCloud.from_file(pcl1_path)
pc2 = LidarPointCloud.from_file(pcl2_path)

In [ ]:
# Get set of object centers

_, boxes0, _= nusc.get_sample_data(lidar_top_data_0['token'], selected_anntokens=None, use_flat_vehicle_coordinates =False)
_, boxes1, _= nusc.get_sample_data(lidar_top_data_1['token'], selected_anntokens=None, use_flat_vehicle_coordinates =False)
_, boxes2, _= nusc.get_sample_data(lidar_top_data_2['token'], selected_anntokens=None, use_flat_vehicle_coordinates =False)
# for box in boxes:
    # c = np.array(self.get_color(box.name)) / 255.0
    # box.render(axes[0], view=view, colors=(c, c, c))
    # corners = view_points(boxes[0].corners(), view, False)[:2, :]

In [ ]:
# Load point clouds
pc0.points

print("rows:",np.size(pc0.points,0))
print("columns:",np.size(pc0.points,1))


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d
# Plot 3D pointcloud
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(pc0.points[0], pc0.points[1], pc0.points[2])
plt.show()


In [ ]:
# test out Boxes
print(boxes0[0])
print(type(boxes0[0].center))
print(type(boxes0[0].wlh))
print(type(boxes0[0].orientation.angle))


In [ ]:
from visualization.open3d_vis import Visualizer
# # visualization
pc0_transposed = np.transpose(pc0.points)
print("Shape:",np.shape(pc0_transposed))


boxes_kitti_style = []
for box in boxes0:
    boxes_kitti_style.append(box.center.tolist() + box.wlh.tolist() + [box.orientation.angle])

boxes_kitti_style = np.asarray(boxes_kitti_style)
print(boxes_kitti_style[0:1])
print(type(boxes_kitti_style[0:1]))
print("Shape:",np.shape(boxes_kitti_style))

print(boxes_kitti_style[0:1,0:3])

In [ ]:

# #Add Pointcloud to visualizer
# results = Visualizer(pc0_transposed, center_mode='lidar_bottom',points_size=3)
# #Add Bounding Boxes
# results.add_bboxes(boxes_kitti_style, bbox_color=[0.9, 0.2, 0])
# results.show(save_path=None)


In [ ]:
# Get set of object centers
# Stack Boxes/Centroids
from utility import get_box_centers

centers0 = get_box_centers(boxes0)
centers1 = get_box_centers(boxes1)
centers2 = get_box_centers(boxes2)

# Testing numpy commands
# print(centers0[0:3, 0:2])
# print(centers0.shape[1])
# print(np.ones((centers0.shape[1],centers0.shape[1])))
# centers0 @ np.ones((centers0.shape[1],centers0.shape[1]))

# Special Shift parameter in meter
l = 20
# Boxes 0 can stay at the current frame
# centers0
# Boxes 1 must be translated up by l meters
print(centers1[0])
centers1 += np.array([0,0,l])
print(centers1[0])
# Boxes 2 must be translated up by 2*l meters
centers2 += np.array([0,0,2*l])

# Add all centroids into one array
centers = np.empty((0,3))
centers = np.append(centers, centers0, axis=0)
centers = np.append(centers, centers1, axis=0)
centers = np.append(centers, centers2, axis=0)

assert centers.shape[0] == centers0.shape[0] \
            + centers1.shape[0] + centers2.shape[0]

print(centers[0:4])

In [ ]:
fig_centers = plt.figure()
ax = fig_centers.add_subplot(111,projection='3d')

# ax.scatter(centers[:,0], centers[:,1], centers[:,2])
ax.scatter(centers0[:,0], centers0[:,1], centers0[:,2])
ax.scatter(centers1[:,0], centers1[:,1], centers1[:,2])
ax.scatter(centers2[:,0], centers2[:,1], centers2[:,2])

In [ ]:
#Add Pointcloud to visualizer
# results = Visualizer(centers, center_mode='lidar_bottom',points_size=3)
# results.show(save_path=None)


In [ ]:
# Build KNN Graph
from sklearn.neighbors import NearestNeighbors
# Init NN-object to handle distance meassurements and other operations
nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(centers)
# Compute Distances and indices of k center-entries that lie the nearest to current centroids
distances, indices = nbrs.kneighbors(centers)
print("Indices:\n",indices[0:3],"Shape: ",indices.shape)
print("Distances:\n",distances[0],"Shape: ",distances.shape)

# Show connectivity of graph
print(nbrs.kneighbors_graph(centers).toarray())


In [ ]:
#Build graph in a more organised manner
from graph.graph_generation import SpatioTemporalGraph, Timeframe

nbrs_0 = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(centers0)

# Add spatial edges #####################

# TODO NearestNeighbors includes the current node. 
#       Change it so that nodes do not point at themselves

# Frame t0
#Compute K nearest neighbors
spatial_distances_0, spatial_indices_0=nbrs_0.kneighbors(centers0)
# Make a list of tuple pairs
spatial_pairs = [] 
for curr_node_idx ,neigborhood_indices,  in enumerate(spatial_indices_0):
    for neigbor_index in neigborhood_indices:
        spatial_pairs.append( \
            ( (Timeframe.t0,curr_node_idx) , ((Timeframe.t0,neigbor_index)) ) )

#Frame t1
nbrs_1 = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(centers1)
spatial_distances_1, spatial_indices_1=nbrs_1.kneighbors(centers1)
# Make a list of tuple pairs
for curr_node_idx ,neigborhood_indices,  in enumerate(spatial_indices_1):
    for neigbor_index in neigborhood_indices:
        spatial_pairs.append( \
            ( (Timeframe.t1,curr_node_idx) , ((Timeframe.t1,neigbor_index)) ) )

#Frame t2
nbrs_2 = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(centers2)
spatial_distances_2, spatial_indices_2=nbrs_2.kneighbors(centers2)
# Make a list of tuple pairs
for curr_node_idx ,neigborhood_indices,  in enumerate(spatial_indices_2):
    for neigbor_index in neigborhood_indices:
        spatial_pairs.append( \
            ( (Timeframe.t2,curr_node_idx) , ((Timeframe.t2,neigbor_index)) ) )

testgraph = SpatioTemporalGraph(spatial_pairs)


In [ ]:
from utility import is_same_instance

print(nusc.get('sample_annotation', boxes0[0].token))

print("Same Instances: ",is_same_instance(nusc, boxes0[0].token,boxes0[0].token))
print("Different Instances:",is_same_instance(nusc, boxes0[0].token,boxes0[1].token))

In [ ]:
# Add temporal edges #####################
# Only connect unique objects to themselves. Only Connect boxes
# that belong to the same object instance and subsequently same class
#Initiate graph
temporal_pairs = []
# Looking for connections between frame i and frame j
# Frame 0 ==> Frame 1
for i,box_i in enumerate(boxes0):
    for j,box_j in enumerate(boxes1):
        if is_same_instance(nusc, box_i.token, box_j.token):
            temporal_pairs.append( \
                ( (Timeframe.t0,i) , ((Timeframe.t1,j)) ) )
print(np.shape(temporal_pairs))
# Frame 1 ==> Frame 2
for i,box_i in enumerate(boxes1):
    for j,box_j in enumerate(boxes2):
        if is_same_instance(nusc, box_i.token, box_j.token):
            temporal_pairs.append( \
                ( (Timeframe.t1,i) , ((Timeframe.t2,j)) ) )
print(np.shape(temporal_pairs))
print(temporal_pairs[0])

testgraph.add_connections(temporal_pairs)


In [ ]:
print("Time 0:\n",testgraph._graph[(Timeframe.t0,0)])
print("Time 1:\n",testgraph._graph[(Timeframe.t1,0)])
print("Time 2:\n",testgraph._graph[(Timeframe.t2,0)])
# print("Whole Graph\n",testgraph._graph)

In [ ]:
# Get all spatial Edges
spatial_pointpairs0 = []
for reference_node in testgraph._graph:
    if(reference_node[0]== Timeframe.t0):
        for neighbor_node in testgraph._graph[reference_node]:
            # print(neighbor_index[0])
            timestep, idx = neighbor_node[0],neighbor_node[1]
            if timestep == Timeframe.t0:
                spatial_pointpairs0.append([reference_node[1],idx])

print(np.shape(spatial_pointpairs0))
testarray = testgraph.get_spatial_pointpairs(Timeframe.t0)
assert spatial_pointpairs0 == testarray

spatial_pointpairs1 = testgraph.get_spatial_pointpairs(Timeframe.t1)
spatial_pointpairs2 = testgraph.get_spatial_pointpairs(Timeframe.t2)

In [ ]:
np.argwhere(centers == centers0[-1])
np.argwhere(centers == centers1[-1])[0,0]

In [ ]:
# Get all temporal edges

# Get points 
def get_points(centers0, centers1 ,centers2,reference_node):
    if(reference_node[0]== Timeframe.t0):
        return centers0[reference_node[1]]
    elif (reference_node[0]== Timeframe.t1):
        return centers1[reference_node[1]]
    elif (reference_node[0]== Timeframe.t2):
        return centers2[reference_node[1]]
    else:
        return AttributeError
        
# Get all temporal edges (only frame 0->1 and 1->2)
# global_center_list = centers.tolist()
temporal_pairs_indices = []
for reference_node in testgraph._graph:
    reference_timeframe = reference_node[0]

    # Find corresponding indices in global centers list
    point_a = get_points(centers0, centers1 ,centers2,reference_node)
    reference_idx_global = np.argwhere(centers == point_a)[0,0]

    for neighbor_node in testgraph._graph[reference_node]:
        # print(neighbor_index[0])
        neighbor_timeframe, neighbor_idx = neighbor_node[0],neighbor_node[1]
        if neighbor_timeframe != reference_timeframe:
            # Find corresponding indices in global centers list
            # Reference Node
            point_b = get_points(centers0, centers1 ,centers2,neighbor_node)
            neighbor_idx_global = np.argwhere(centers == point_b)[0,0]
            #Append global indices into list
            temporal_pairs_indices.append([reference_idx_global,neighbor_idx_global])


In [ ]:
import open3d as o3d
from open3d import geometry

############ Parameters
upward_shift = 20
##############

# Integrate All Center points in Pointcloud-object not necessary but test
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(centers)

# Include Pointcloud of timeframe 0 into visualization 
pc0_transposed = np.transpose(pc0.points)
print("Shape:",np.shape(pc0_transposed))
pointcloud0 = o3d.geometry.PointCloud()
pointcloud0.points = o3d.utility.Vector3dVector(pc0_transposed[:, :3])
#Add some color
point_color=(0.5, 0.5, 0.5)
points_colors = np.tile(np.array(point_color), (pc0_transposed.shape[0], 1))
pointcloud0.colors = o3d.utility.Vector3dVector(points_colors)

# Include Pointcloud of timeframe 1 into visualization 
pc1_transposed = np.transpose(pc1.points)
print("Shape:",np.shape(pc1_transposed))
pointcloud1 = o3d.geometry.PointCloud()
pointcloud1.points = o3d.utility.Vector3dVector(pc1_transposed[:, :3])
#Add some color
point_color=(0.5, 0.5, 0.5)
points_colors = np.tile(np.array(point_color), (pc1_transposed.shape[0], 1))
pointcloud1.colors = o3d.utility.Vector3dVector(points_colors)
# Translate up to stack the point clouds
pointcloud1.translate(np.array([0,0,upward_shift]))

# Include Pointcloud of timeframe 2 into visualization 
pc2_transposed = np.transpose(pc2.points)
print("Shape:",np.shape(pc2_transposed))
pointcloud2 = o3d.geometry.PointCloud()
pointcloud2.points = o3d.utility.Vector3dVector(pc2_transposed[:, :3])
#Add some color
point_color=(0.5, 0.5, 0.5)
points_colors = np.tile(np.array(point_color), (pc2_transposed.shape[0], 1))
pointcloud2.colors = o3d.utility.Vector3dVector(points_colors)
# Translate up to stack the point clouds
pointcloud2.translate(np.array([0,0,upward_shift * 2]))

# Include reference frame
mesh_frame = geometry.TriangleMesh.create_coordinate_frame(
            size=1, origin=[0, 0, 0])  # create coordinate frame

# Draw Graph/Edges with Lineset
# Spatial Edges Red Edges
colors = [[1, 0, 0] for i in range(len(spatial_pointpairs0))]
line_set0 = geometry.LineSet(points=o3d.utility.Vector3dVector(centers0),
    lines=o3d.utility.Vector2iVector(spatial_pointpairs0),)
line_set0.colors = o3d.utility.Vector3dVector(colors)

colors = [[1, 0, 0] for i in range(len(spatial_pointpairs1))]
line_set1 = geometry.LineSet(points=o3d.utility.Vector3dVector(centers1),
    lines=o3d.utility.Vector2iVector(spatial_pointpairs1),)
line_set1.colors = o3d.utility.Vector3dVector(colors)

colors = [[1, 0, 0] for i in range(len(spatial_pointpairs2))]
line_set2 = geometry.LineSet(points=o3d.utility.Vector3dVector(centers2),
    lines=o3d.utility.Vector2iVector(spatial_pointpairs2),)
line_set2.colors = o3d.utility.Vector3dVector(colors)

# Temporal Edges 
colors = [[0, 0, 1] for i in range(len(temporal_pairs_indices))]
temporal_line_set = geometry.LineSet(points=o3d.utility.Vector3dVector(centers),
    lines=o3d.utility.Vector2iVector(temporal_pairs_indices),)
temporal_line_set.colors = o3d.utility.Vector3dVector(colors)



In [ ]:
# help(o3d.visualization.draw_geometries)
o3d.visualization.draw_geometries([pcd,mesh_frame,line_set0, \
            line_set1,line_set2,temporal_line_set,\
            pointcloud0,pointcloud1, pointcloud2])

# o3d.visualization.draw_geometries([pcd,mesh_frame,line_set0,pointcloud0,pointcloud1, pointcloud2])
# o3d.visualization.draw_geometries([pcd,mesh_frame,line_set,pointcloud0,pointcloud1],
#                                   zoom=0.3412,
#                                   front=[0.4257, -0.2125, -0.8795],
#                                   lookat=[2.6172, 2.0475, 1.532],
#                                   up=[-0.0694, -0.9768, 0.2024])

In [ ]:
# help(pointcloud1.translate)
# help(o3d.visualization.Visualizer.add_geometry())

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(centers)

# from visualization.open3d_vis import _draw_bboxes
# boxes_kitti_style = []
# for box in boxes0:
#     boxes_kitti_style.append(box.center.tolist() + box.wlh.tolist() + [box.orientation.angle])

# boxes_kitti_style = np.asarray(boxes_kitti_style)
# print(boxes_kitti_style[0:1])
# print(type(boxes_kitti_style[0:1]))
# print("Shape:",np.shape(boxes_kitti_style))

# print(boxes_kitti_style[0:1,0:3])

# Include reference frame
mesh_frame = geometry.TriangleMesh.create_coordinate_frame(
            size=1, origin=[0, 0, 0])  # create coordinate frame

vis = o3d.visualization.Visualizer()
vis.create_window()

vis.get_render_option().point_size = 5
vis.add_geometry(pcd)
vis.add_geometry(mesh_frame)
vis.add_geometry(line_set0)

# boxes0
# _draw_bboxes(vis=vis,pcd=pcd)

vis.run()
vis.destroy_window()

In [ ]:
# o3d.visualization.Visualizer.add_geometry()
def custom_draw_geometry_with_rotation(pcd):

    def rotate_view(vis):
        ctr = vis.get_view_control()
        ctr.rotate(10.0, 0.0)
        return False

    o3d.visualization.draw_geometries_with_animation_callback([pcd],
                                                              rotate_view)

custom_draw_geometry_with_rotation(pcd)